In [22]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_dataset
import pandas as pd
import numpy as np
from src.tfr_decoding.finepref_sample import sample # new sampling method
from src.utils.samp_utils import inpsampall, dset_randsamp   
from src.tfr_decoding.shp_modeling import T5BinaryClassifier
from prefix_sampling import PrefixSampler, test_baseline, test_pfsample, test_finesample, test_apsample

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# first load relevant models
device = 'cuda:1' # if you have a GPU


# get generation model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
qpref = T5BinaryClassifier.load_from_checkpoint("./lightning_logs/version_4/checkpoints/epoch=2-step=11896.ckpt")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")#.to(device)
#model.sample = sample.__get__(model)
model.tokenizer = tokenizer
model.tok = tokenizer
pfname = 'stanfordnlp/SteamSHP-flan-t5-large'
max_len = 512
learning_rate = 3e-5
preftok = T5Tokenizer.from_pretrained(pfname)

model.qualitypref = qpref
#model.stdpref = T5BinaryClassifier.load_from_checkpoint("./lightning_logs/version_11/checkpoints/epoch=3-step=3359.ckpt")
# set up relevant mdoels
model.downmetric = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)
model.downtok = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
model.eval()

# load up our dataset
elidf = pd.read_json("output/elidataset.jsonl", orient="records", lines="true")
# start from 15000 for latest exploration
elidf = elidf.drop_duplicates(subset="history").iloc[15000:15100]

pfsampler = PrefixSampler(model)
inplist = list(elidf['history'])

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
torch.backends.cuda.enable_flash_sdp(True)

In [24]:
finesample_df = test_apsample(inplist[:10], pfsampler, 0.85, 3, 1, 3)


monkeysamp
-
-
[tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0])]
[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12]
tensor([0.6804], device='cuda:1')   Whats the current status of the protests in Syria? Who is the president? What is the government doing to keep them down? Do they really need the US to help?
monkeysamp
-
-
-
-
[tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1])]
[-1, -2, -3, -4, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
tensor([0.7525], device='cuda:1')   I am a little confused on Syria but here is the information I have: President Bashar al-Assad's response to the unrest began on March 18 with a crackdown on protesters that turned violent as the S

In [20]:
finesample_df['budgets'].mean()

137.4

In [3]:
a1 = pd.read_json("output/pfsample/finesamp.jsonl", orient="records", lines=True)


In [5]:
a1.budgets.mean()

115.71

In [ ]:
adaptbase_df = test_baseline(inplist, pfsampler, .85, 4)
pfsample_df = test_pfsample(inplist, pfsampler, 18, [10, 20])

adaptbase_df.to_json("output/pfsample/abase.jsonl", lines=True, orient='records')
pfsample_df.to_json("output/pfsample/pfsample.jsonl", lines=True, orient='records')

monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp


In [8]:
print(adaptbase_df['budgets'].mean())
print(adaptbase_df['scos'].mean())

130.4
0.809410572052002


In [12]:
print(pfsample_df['budgets'].mean())
print(pfsample_df['scos'].mean())

253.6
0.8504377722740173


In [26]:
adaptbase_df = test_baseline(inplist, pfsampler, .85, 2)

monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp


In [27]:
print(adaptbase_df['budgets'].mean())
print(adaptbase_df['scos'].mean())

129.2
0.8531928896903992
